In order to figure out which experiments are worth pursuing, **experiment tracking** comes in

In this notebook, example of programatically tracking experiment can be seen


In [ ]:
import torch
import torchvision
from torch import nn

print(torch.__version__)
print(torchvision.__version__)

In [ ]:
'''
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
'''

In [ ]:
'''
# regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    # Clones github repo
    # moves specified directory to root with '.'
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/aayush2058/pytorch_fundamentals
    !move pytorch_fundamentals/going_modular .
'''

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
def set_seeds(seed: int = 42):
    '''
    Sets random sets for torch operations.
    
    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    '''
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

### Get data

In [ ]:
# Food vision dataset (pizza, steak, sushi)
import os
import zipfile

from pathlib import Path

import requests

def download_zipped_data(source: str,
                        destination: str,
                        remove_source: bool = True) -> Path:
    """
    Downloads a zipped dataset from source and unzips to destination.
    """
    # Setup path to a data folder
    data_path = Path("data/")
    image_path = data_path/destination
    
    # If the image folder doesn't exist, create it
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory already exists, skipping download.")
        
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents = True, exist_ok = True)
        
        # Download the target data
        target_file = Path(source).name # this takes title of the file
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)
            
        # Unzip target file
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...")
            zip_ref.extractall(image_path)
            
        # Remove .zip file if needed
        if remove_source:
            os.remove(data_path / target_file)
            
    return image_path

In [ ]:
data_path = Path("data/")
image_path = download_zipped_data(source = "https://github.com/aayush2058/Food-identification-PyTorch-/raw/main/data/pizza_steak_sushi.zip",
                    destination = "pizza_steak_sushi")

In [ ]:
image_path

#### Walking through data

In [ ]:
for dir_path, dir_names, filenames in os.walk(image_path):
    print(f"There are {len(dir_names)} directories and {len(filenames)} images in '{dir_path}'.")


### Datasets and DataLoaders


In [ ]:
# Setup directories
train_dir = image_path/"train"
test_dir = image_path/"test"

train_dir, test_dir

In [ ]:
# Creating manual transformer with normalization labels
from going_modular import data_setup
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_transformer = transforms.Compose([
                                 transforms.Resize(size = (256, 256)),
                                 transforms.CenterCrop(size = [224]),
                                 transforms.ToTensor(),
                                 normalize])

print(f"Manual transform setup: {manual_transformer}")

# Creating dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir = train_dir,
                                                                              test_dir = test_dir,
                                                                              train_transform = manual_transformer,
                                                                              test_transform = manual_transformer, 
                                                                              batch_size = 32)
train_dataloader, test_dataloader, class_names

In [ ]:
# Creating automatic transformer
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Creating automatic transformer
automatic_transformer = weights.transforms()

print(f"Automatic transform setup: {automatic_transformer}")


# Creating dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir = train_dir,
                                                                              test_dir = test_dir,
                                                                              train_transform = automatic_transformer,
                                                                              test_transform = automatic_transformer,
                                                                              batch_size = 32)
train_dataloader, test_dataloader, class_names

### Getting a model

In [ ]:
# getting up to date default weights for this model
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# creating model with default weights
model = torchvision.models.efficientnet_b0(weights = weights).to(device)
model.to(device)

In [ ]:
'''
Freeze all the base layers by setting their requires_grad attribute to False.
Because PyTorch automatically tracks the gradients of our model parameters and if we want to update them, optimizer do so.
So we set requires_grad = False so that those parameters are freezed while training.
'''
for param in model.features.parameters():
    # print(param)
    param.requires_grad = False   

In [ ]:
# Change the classifier head
print(f"Default classifier of the model: {model.classifier}")
model.classifier = nn.Sequential(
                                nn.Dropout(p = 0.2, inplace = True),
                                nn.Linear(in_features = 1280,
                                         out_features = len(class_names))).to(device)

print(f"\nModified classifier for our problem: {model.classifier}")

In [ ]:
from torchinfo import summary

summary(model = model,
       input_size = (32, 3, 224, 224),
       verbose = 0,
       col_names = ["input_size", "output_size", "num_params", "trainable"],
       col_width = 20,
       row_settings = ["var_names"])

#### Train a single model and track resutls

In [ ]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                            lr = 0.001)

In [ ]:
# Setup a summary writer
import tensorboard
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
writer

In [ ]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from going_modular.engine import train_step, test_step

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        ##### NEW: Experiment tracking #####
        # open writer(SummaryWriter())
        writer.add_scalars(main_tag = "Loss",
                          tag_scalar_dict = {"train_loss": train_loss,
                                             "test_loss": test_loss},
                          global_step = epoch)
        
        writer.add_scalars(main_tag = "Accuracy",
                          tag_scalar_dict = {"train_acc": train_acc,
                                             "test_acc": test_acc},
                          global_step = epoch)
        
        writer.add_graph(model = model,
                        input_to_model = torch.randn(32, 3, 224, 224).to(device)
                        )
    # close writer
    writer.close()
    ##### End NEW #####

    # Return the filled results at the end of the epochs
    return results


In [ ]:
results_1 = train(model = model, 
      train_dataloader = train_dataloader, 
      test_dataloader = test_dataloader, 
      optimizer = optimizer,
      loss_fn = loss_fn,
      epochs = 5,
      device = device)

#### View model's results with Tensorboard

In [ ]:
# Viewing experiments from within the notebook
import tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

### create a functin to prepare a `SummaryWriter()` instance

By default our `SummaryWriter()` class saves to `log_dir`.

How about if we wanted to save different experiments to different folders?

In essence, one experiment = one folder
example:
   * Experiment data/timestamp
   * Experiment name
   * Model name
   * Extra - is there anything else that should be tracked?
    
Let's create a function to create a `SummaryWriter()` instance to take all of these things into account.

So ideally we end up tracking experiments to a directory:

`runs/YYTT-MM-DD/experiment_name/model/model_name/extra`

In [ ]:
from torch.utils.tensorboard import SummaryWriter

def create_writer(experiment_name: str,
                 model_name: str,
                 extra: str = None):
    """
    Creates a torch.utils.tensorboard.writer.SummaryWriter() instance tracking to a specific directory.
    """
    from datetime import datetime
    import os
    
    # Get timestamp of current date in reverse order
    timestamp = datetime.now().strftime("%Y-%m-%d")
    
    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
    print(f"[INFO] Created SummaryWriter saving to {log_dir}")
    return SummaryWriter(log_dir = log_dir)

In [ ]:
example_writer = create_writer(experiment_name = "data_10_percent",
                              model_name = 'effnetb0',
                              extra = "5_epochs")
example_writer

### Update the `train()` function to include a `writer` parameter
Below function can track multiple experiments if we need to

In [ ]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from going_modular.engine import train_step, test_step

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter = None) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        ##### NEW: Experiment tracking #####
        if writer:
            # open writer(SummaryWriter())
            writer.add_scalars(main_tag = "Loss",
                              tag_scalar_dict = {"train_loss": train_loss,
                                                 "test_loss": test_loss},
                              global_step = epoch)

            writer.add_scalars(main_tag = "Accuracy",
                              tag_scalar_dict = {"train_acc": train_acc,
                                                 "test_acc": test_acc},
                              global_step = epoch)

            writer.add_graph(model = model,
                            input_to_model = torch.randn(32, 3, 224, 224).to(device)
                            )
            # close writer
            writer.close()
            ##### End NEW #####
        else:
            pass

    # Return the filled results at the end of the epochs
    return results


### Setting up a series of model experiments

Example: 
Setup two procedures for same dataset and train for different epochs value to track experiments between two procedures.

In [ ]:
writer_1 = create_writer(experiment_name = "data_10_percent",
                              model_name = 'effnetb0',
                              extra = "5_epochs_SGD_optimizer")

In [ ]:
test_1 = train(model = model, 
          train_dataloader = train_dataloader, 
          test_dataloader = test_dataloader, 
          optimizer = torch.optim.SGD(model.parameters(), lr = 0.01),
          loss_fn = loss_fn,
          epochs = 5,
          device = device,
          writer = writer_1)

In [ ]:
from helper_functions import plot_loss_curves
plot_loss_curves(test_1)

In [ ]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

### To compare different models and run limitless experiments,

* Change the number of epochs
* Change the number of hidden layers/units
* Change the amount of data (right now we're using 10% of the Food101 dataset for pizza, steak, sushi)
* Change the learning rate
* Try different kinds of data augmentation
* Choose a different model architecture



### What experiments are we going to run?

We're going to turn three dials:
1. Model size - EfficientNetB0 vs EfficientNetB2
2. Dataset size - 10% of pizza, steak, sushi images vs 20% (generally, more data = better results)
3. Training time - 5 epochs vs 10 epochs (generally longer training time = better results, up to a point)

To begin, we're still keeping things relatively small so that our experiments run quickly.

**Our goal:** a model that is well performing but still enough to run on a mobile device or web browser, so FoodVision Mini can come to life.

If you had infinite compute + time, you should basically always choose the biggest model and biggest dataset you can.

## Another project workflow starts

#### Download different datasets

We want two datasets:
1. pizza_steak_sushi 10%
2. pizza_steak_sushi 20%


In [ ]:
# Download zipped data and extract into the directory

pizza_steak_suushi_10 = download_zipped_data(source = "https://github.com/aayush2058/pytorch_fundamentals/raw/main/data/pizza_steak_sushi.zip",
                                            destination = "pizza_steak_sushi_10",
                                            remove_source = True)


pizza_steak_suushi_20 = download_zipped_data(source = "https://github.com/aayush2058/pytorch_fundamentals/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                            destination = "pizza_steak_sushi_20",
                                            remove_source = True)


In [ ]:
# image path for 20%_data
image_path_10 = Path("data/pizza_steak_sushi_10")
image_path_10

# image path for 20%_data
image_path_20 = Path("data/pizza_steak_sushi_20")
image_path_20

In [ ]:
# Walking thorugh pizza_stek_sushi_10%

for dir_path, dir_names, filenames in os.walk(image_path_10):
    print(f"There are {len(dir_names)} directories and {len(filenames)} images in '{dir_path}'.")


In [ ]:
# Walking through pizza_steak_sushi_20%

for dir_path, dir_names, filenames in os.walk(image_path_20):
    print(f"There are {len(dir_names)} directories and {len(filenames)} images in '{dir_path}'.")


In [ ]:
# For 10% data and 20% data
train_dir_10 = image_path_10/"train"
train_dir_20 = image_path_20/"train"

# Test dir remains same to compare between the performance of models
test_dir = image_path_10/"test"

### Dataloaders

In [ ]:
# Creating automatic transformer
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Creating automatic transformer
automatic_transformer = weights.transforms()

print(f"Automatic transform setup: {automatic_transformer}")


BATCH_SIZE = 32
# Creating dataloaders for 10%
train_dataloader_10, test_dataloader_10, class_names_10 = data_setup.create_dataloaders(train_dir = train_dir_10,
                                                                              test_dir = test_dir,
                                                                              train_transform = automatic_transformer,
                                                                              test_transform = automatic_transformer,
                                                                              batch_size = BATCH_SIZE)

train_dataloader_20, test_dataloader_20, class_names_20 = data_setup.create_dataloaders(train_dir = train_dir_20,
                                                                              test_dir = test_dir,
                                                                              train_transform = automatic_transformer,
                                                                              test_transform = automatic_transformer,
                                                                              batch_size = BATCH_SIZE)

train_dataloader_10, test_dataloader_10, class_names_10, train_dataloader_20, test_dataloader_20, class_names_20

In [ ]:
print(f"Number of batches of size {BATCH_SIZE} in 10% train data: {len(train_dataloader_10)}")
print(f"Number of batches of size {BATCH_SIZE} in 20% train data: {len(train_dataloader_20)}")

### Models

In [ ]:
# Effnetb2
# getting up to date default weights for this model
weights_efnetb0 = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# creating model with default weights
effnetb0 = torchvision.models.efficientnet_b0(weights = weights_efnetb0).to(device)
effnetb0.to(device)

# -------------------------------------------------------------------

# Effnetb2
# getting up to date default weights for this model
weights_efnetb2 = torchvision.models.EfficientNet_B2_Weights.DEFAULT

# creating model with default weights
effnetb2 = torchvision.models.efficientnet_b2(weights = weights_efnetb2).to(device)
effnetb2.to(device)


In [ ]:
summary(model = effnetb0,
       input_size = (32, 3, 224, 224),
       verbose = 0,
       col_names = ["input_size", "output_size", "num_params", "trainable"],
       col_width = 20,
       row_settings = ["var_names"])

In [ ]:
summary(model = effnetb2,
       input_size = (32, 3, 224, 224),
       verbose = 0,
       col_names = ["input_size", "output_size", "num_params", "trainable"],
       col_width = 20,
       row_settings = ["var_names"])

In [ ]:
# Freeze base model layers

set_seeds(seed = 42)

for param in effnetb0.features.parameters():
    param.requires_grad = False
    
for param in effnetb2.features.parameters():
    param.requires_grad = False
    
# Change classifier head
effnetb0.classifier = nn.Sequential(
                                    nn.Dropout(p=0.3, inplace=True),
                                    nn.Linear(in_features=1280, 
                                           out_features=len(class_names_10))
                                    )

effnetb2.classifier = nn.Sequential(
                                    nn.Dropout(p=0.3, inplace=True),
                                    nn.Linear(in_features=1408, 
                                           out_features=len(class_names_10))
                                    )
    
    
    

In [ ]:
summary(model = effnetb0,
       input_size = (32, 3, 224, 224),
       verbose = 0,
       col_names = ["input_size", "output_size", "num_params", "trainable"],
       col_width = 20,
       row_settings = ["var_names"])

#### Create experiments and setup training code

In [ ]:
# Create epoch list
num_epochs = [5, 10]

# Create models list (need to create a new model for each experiment)
models = ["effnetb0", "effnetb2"]

# Create a DataLoaders dictionary
train_dataloaders = {"data_10_percent": train_dataloader_10,
                    "data_20_percent": train_dataloader_20}

In [ ]:
%%time
from going_modular.utils import save_full_model

# Set seeds
set_seeds(seed = 42)

# Keep track of experiment numbers
experiment_number = 0

# Loop through each DataLoader
for dataloader_name, train_dataloader in train_dataloaders.items():
    # Loop through the epochs
    for epochs in num_epochs:
        # Loop through each model name and create a new model instance
        for model_name in models:
            
            # Print out info
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")
            
            # Select and create the model
            if model_name == "effnetb0":
                model = effnetb0
            else:
                model = effnetb2
                
            # Create a new loss and optimizer for every experiment
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params = model.parameters(),
                                        lr = 0.001)
            
            # Train the target model with target dataloader and track experiments
            # Note: Using above modified train function which includes writer parameter.
            train(model = model,
                 train_dataloader = train_dataloader,
                 test_dataloader = test_dataloader,
                 optimizer = optimizer,
                 loss_fn = loss_fn,
                 epochs = epochs,
                 device = device, 
                 writer = create_writer(experiment_name = dataloader_name,
                                       model_name = model_name,
                                       extra = f"{epochs}_epochs"))
            
            # Save the model to file so we can import it later if need be
            save_filepath = f"{model_name}_{dataloader_name}_{epochs}.epochs.pth"
            save_full_model(model = model,
                      target_dir = "models",
                      model_name = save_filepath)
            print("-" * 50 + "\n")

In [ ]:
# View summary
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

##### The best performing model was:
   * EffNetB2
   * pizza, steak, sushi 20%
   * epochs: 10   

## We can share our tensorboad experiment tracking with the help of tensorboard dev

Below code is commented out because it is not responding to my current session. This structure can be used to share experiments if necessary

In [ ]:
# ## Upload the results to TensorBoard.dev
# !tensorboard dev upload --logdir runs \
#     --name " PyTorch Experiment Tracking: Food Identification model result (1st try)" \
#     --description " Comparing results of different model size, amount of training data and training time."


### Load in the best model and make prediction with it

In [ ]:
# Setup best model filepath
best_model_path = "models/effnetb2_data_20_percent_10.epochs.pth"

# Load the saved model
best_model = torch.load(best_model_path)

# If the model dictionary needs to be loaded, we need to instantiate the model with same amount of features and input/output shape

###### The goal is to create a foodIdentify model that performs well enough and is able to run on a mobile device / web browser

In [ ]:
# Check the model file size
from pathlib import Path

# Get the model size in bytes then convert it to megabytes
effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
print(f"EfficinetNetB2 feature extractor model size: {effnetb2_model_size} MB")


In [ ]:
# Import functions to make prediction on images and plot them
from going_modular.predictions import pred_and_plot_image

# Get a random list of 3image path names from the test dataset
import random
num_images_to_plot = 3
test_image_path_list = list(Path(image_path_20/ "test").glob("*/*.jpg"))
test_image_path_sample = random.sample(test_image_path_list,
                                      k = num_images_to_plot)
for image_path in test_image_path_sample:
    pred_and_plot_image(model = best_model,
                       image_path = image_path,
                       class_names = class_names,
                       image_size = (224, 224))

#### Testing model on a custom image

In [ ]:
# Downloading custom image
# Download the image
import requests

# Setup custom image path
custom_image_path1 = Path("data/hidden_pizza.png")

# Download the image if it doesn't exist (hidden_pizza.png)
if not custom_image_path1.is_file():
    with open(custom_image_path1, "wb") as f:
        request1 = requests.get("https://github.com/aayush2058/Food-identification-PyTorch-/raw/main/data/hidden_pizza.png")
        print(f"Downloading {custom_image_path1}.... ")
        f.write(request1.content)
        print("successful") 
        
else:
    print(f"{custom_image_path1} already exists, skipping download...")

In [ ]:
# Downloading custom image
import requests

# Setup custom image path
custom_image_path2 = Path("data/sushi_served.png")
        
# Download the image if it doesn't exist (sushi_served.png)
if not custom_image_path2.is_file():
    with open(custom_image_path2, "wb") as f:
        request2 = requests.get("https://github.com/aayush2058/Food-identification-PyTorch-/raw/main/data/sushi_served.png")
        print(f"Downloading {custom_image_path2}.... ")
        f.write(request2.content)
        print("successful")   
else:
    print(f"{custom_image_path2} already exists, skipping download...")

In [ ]:
"""
Utility functions to make predictions.

Main reference for code creation: https://www.learnpytorch.io/06_pytorch_transfer_learning/#6-make-predictions-on-images-from-the-test-set 
"""
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

from typing import List, Tuple

from PIL import Image

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Predict on a target image with a target model
# Function created in: https://www.learnpytorch.io/06_pytorch_transfer_learning/#6-make-predictions-on-images-from-the-test-set
def pred_and_plot_image(
    model: torch.nn.Module,
    class_names: List[str],
    image_path: str,
    image_size: Tuple[int, int] = (224, 224),
    transform: torchvision.transforms = None,
    device: torch.device = device,
):
    """Predicts on a target image with a target model.

    Args:
        model (torch.nn.Module): A trained (or untrained) PyTorch model to predict on an image.
        class_names (List[str]): A list of target classes to map predictions to.
        image_path (str): Filepath to target image to predict on.
        image_size (Tuple[int, int], optional): Size to transform target image to. Defaults to (224, 224).
        transform (torchvision.transforms, optional): Transform to perform on image. Defaults to None which uses ImageNet normalization.
        device (torch.device, optional): Target device to perform prediction on. Defaults to device.
    """

    # Open image
    img = Image.open(image_path).convert('RGB')

    # Create transformation for image (if one doesn't exist)
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose(
            [
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    ### Predict on image ###

    # Make sure the model is on the target device
    model.to(device)

    # Turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(img).unsqueeze(dim=0)

        # Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # Convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # Plot image with predicted label and probability
    plt.figure()
    plt.imshow(img)
    plt.title(
        f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    )
    plt.axis(False)



In [ ]:
# Prediction
pred_and_plot_image(model = best_model,
                   image_path = custom_image_path1,
                   class_names = class_names)

In [ ]:
# Prediction
pred_and_plot_image(model = best_model,
                   image_path = custom_image_path2,
                   class_names = class_names)

In [ ]:
# Prediction
pred_and_plot_image(model = best_model,
                   image_path = Path("data/pizza_test.jpg"),
                   class_names = class_names)

In [ ]:
# Prediction
pred_and_plot_image(model = best_model,
                   image_path = Path("data/eating_sushi.jpg"),
                   class_names = class_names)